You will need to install Transformers and  Datasets as well as other dependencies. 

In [1]:
! pip install datasets transformers rouge-score nltk

     |████████████████████████████████| 311 kB 8.2 MB/s 
     |████████████████████████████████| 3.5 MB 69.5 MB/s 
     |████████████████████████████████| 134 kB 98.8 MB/s 
     |████████████████████████████████| 1.1 MB 58.5 MB/s 
     |████████████████████████████████| 212 kB 93.4 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 895 kB 57.6 MB/s 
     |████████████████████████████████| 6.8 MB 59.8 MB/s 
     |████████████████████████████████| 596 kB 85.2 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 80.6 MB/s 
     |████████████████████████████████| 144 kB 77.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,624 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155320 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


# Fine-tuning a pre-trained model on a summarization task

To do that, we can use any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`t5-small`](https://huggingface.co/t5-small) checkpoint. 

In [5]:
model_checkpoint = "t5-small"

## Loading the dataset

We will use a sampled publicly available text summarization dataset based on some news article collected  by `Hermann et al (2015)` (can be downloaded from [here](https://drive.google.com/drive/folders/1CP36015srVw9Q0-1kTwOfAuo4z2t2aKh?usp=sharing)) after doing some preprocessing. This can be easily done with the functions `load_dataset` and `load_metric` for calculating the performance. 

In [7]:
from datasets import load_dataset, load_metric

raw_datasets =load_dataset('csv', data_files={'train': 'train.csv', 'test': '/test.csv', 'val': 'val.csv'})
metric = load_metric("rouge")

Using custom data configuration default-8cc7cca13617b5a6


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8cc7cca13617b5a6/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5000
    })
    val: Dataset({
        features: ['document', 'summary'],
        num_rows: 5000
    })
})

To access an actual element, you need to select a split first, then give an index:

In [9]:
raw_datasets["train"][0]

{'document': "editor 's note : in our behind the scenes series , cnn correspondents share their experiences in covering news and analyze the stories behind the events . here , soledad o'brien takes users inside a jail where many of the inmates are mentally ill . an inmate housed on the `` forgotten floor , '' where many mentally ill inmates are housed in miami before trial . miami , florida -lrb- cnn -rrb- -- the ninth floor of the miami-dade pretrial detention facility is dubbed the `` forgotten floor . '' here , inmates with the most severe mental illnesses are incarcerated until they 're ready to appear in court . most often , they face drug charges or charges of assaulting an officer -- charges that judge steven leifman says are usually `` avoidable felonies . '' he says the arrests often result from confrontations with police . mentally ill people often wo n't do what they 're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become m

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [10]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [11]:
show_random_elements(raw_datasets["train"])

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [12]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.


In [14]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

If you are using one of the five T5 checkpoints we have to prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

In [18]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [19]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [20]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[21603, 10, 6005, 3, 31, 7, 2232, 3, 10, 16, 69, 1187, 8, 8073, 939, 3, 6, 3, 75, 29, 29, 25688, 7, 698, 70, 2704, 16, 6013, 1506, 11, 8341, 8, 1937, 1187, 8, 984, 3, 5, 270, 3, 6, 4199, 14677, 3, 32, 31, 2160, 35, 1217, 1105, 1096, 3, 9, 11796, 213, 186, 13, 8, 16, 11171, 33, 19367, 3, 1092, 3, 5, 46, 16, 5058, 629, 26, 30, 8, 3, 2, 11821, 1501, 3, 6, 3, 31, 31, 213, 186, 19367, 3, 1092, 16, 11171, 33, 629, 26, 16, 1337, 3690, 274, 3689, 3, 5, 1337, 3690, 3, 6, 12215, 26, 9, 3, 18, 40, 52, 115, 18, 3, 75, 29, 29, 3, 18, 52, 52, 115, 18, 1636, 8, 24651, 1501, 13, 8, 1337, 3690, 18, 14677, 15, 7140, 12042, 20, 9174, 3064, 19, 3, 26, 17344, 8, 3, 2, 11821, 1501, 3, 5, 3, 31, 31, 270, 3, 6, 16, 11171, 28, 8, 167, 5274, 2550, 21154, 33, 3, 14736, 15, 4094, 552, 79, 3, 31, 60, 1065, 12, 2385, 16, 1614, 3, 5, 167, 557, 3, 6, 79, 522, 2672, 3991, 42, 3991, 13, 12710, 53, 46, 5502, 1636, 3991, 24, 5191, 3, 849, 1926, 90, 99, 348, 845, 33, 1086, 3, 2, 1792, 179, 3110, 106, 725, 3

In [21]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [22]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [23]:
batch_size = 4
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-sum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

Finally, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [24]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [25]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [26]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


We can now finetune our model by just calling the `train` method:

In [27]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: summary, document.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 50000
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 25000


Epoch,Training Loss,Validation Loss


Saving model checkpoint to t5-small-finetuned-xsum/checkpoint-500
Configuration saved in t5-small-finetuned-xsum/checkpoint-500/config.json
Model weights saved in t5-small-finetuned-xsum/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-xsum/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-xsum/checkpoint-500/special_tokens_map.json


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.177300,1.084538,19.992600,9.349600,17.683400,19.123300,19.000000
2,1.164900,1.079481,20.043000,9.359900,17.769200,19.183200,19.000000


Saving model checkpoint to t5-small-finetuned-xsum/checkpoint-1000
Configuration saved in t5-small-finetuned-xsum/checkpoint-1000/config.json
Model weights saved in t5-small-finetuned-xsum/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-xsum/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-xsum/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to t5-small-finetuned-xsum/checkpoint-1500
Configuration saved in t5-small-finetuned-xsum/checkpoint-1500/config.json
Model weights saved in t5-small-finetuned-xsum/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-xsum/checkpoint-1500/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-xsum/checkpoint-1500/special_tokens_map.json
Saving model checkpoint to t5-small-finetuned-xsum/checkpoint-2000
Configuration saved in t5-small-finetuned-xsum/checkpoint-2000/config.json
Model weights saved in t5-small-finetune

TrainOutput(global_step=25000, training_loss=1.1962815795898438, metrics={'train_runtime': 7104.5361, 'train_samples_per_second': 14.076, 'train_steps_per_second': 3.519, 'total_flos': 2.6782396813541376e+16, 'train_loss': 1.1962815795898438, 'epoch': 2.0})

Now we can evalute our finetuned model on the test data.

In [29]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: summary, document.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 4


{'epoch': 2.0,
 'eval_gen_len': 19.0,
 'eval_loss': 1.0794814825057983,
 'eval_rouge1': 20.043,
 'eval_rouge2': 9.3599,
 'eval_rougeL': 17.7692,
 'eval_rougeLsum': 19.1832,
 'eval_runtime': 355.6721,
 'eval_samples_per_second': 14.058,
 'eval_steps_per_second': 3.514}

The rest is just to save the model in order to use in the future as a predictor.

In [30]:
trainer.save_model("sum_model_")

Saving model checkpoint to sum_model_
Configuration saved in sum_model_/config.json
Model weights saved in sum_model_/pytorch_model.bin
tokenizer config file saved in sum_model_/tokenizer_config.json
Special tokens file saved in sum_model_/special_tokens_map.json
